# Importar librerías

In [ ]:
import numpy as np
import pandas as pd

# Leer datos

In [ ]:
df = pd.read_csv('Resultados.csv')

date = df.values[:,0]
j1 = df.values[:,1]
j2 = df.values[:,2]
j3 = df.values[:,3]
j4 = df.values[:,4]

set1 = df.values[:,5:7]
set2 = df.values[:,7:9]
set3 = df.values[:,9:11]

# Procesar información

In [ ]:
players = ['Ale', 'Alan', 'Lucho', 'Rodrigo', 'Pato', 'Diego', 'Seba T.', 'Matias', 'Seba G.']

score_default = []
for player in players:
    score_default = np.append(score_default,5000)

# Cálculo de puntaje base

In [ ]:
pt_p1 = np.zeros(set1.shape[0])
pt_p2 = np.zeros(set1.shape[0])

for i in range(set1.shape[0]):
    
    sets_p1 = 0
    sets_p2 = 0
    
    #Determinar ganador, si hubo
    if set1[i,0] != set1[i,1]:
        if set1[i,0] >= 6 or set1[i,1] >= 6:
            if set1[i,0] > set1[i,1]:
                sets_p1 = sets_p1 + 1
                sets_p2 = sets_p2 + 0
            elif set1[i,0] < set1[i,1]:
                sets_p1 = 0
                sets_p2 = 1
                
    if set2[i,0] != set2[i,1]:        
        if set2[i,0] >= 6 or set2[i,1] >= 6:
            if set2[i,0] > set2[i,1]:
                sets_p1 = sets_p1 + 1
                sets_p2 = sets_p2 + 0
            elif set2[i,0] < set2[i,1]:
                sets_p1 = sets_p1 + 0
                sets_p2 = sets_p2 + 1
                
    if set3[i,0] != set3[i,1]:         
        if set3[i,0] >= 6 or set3[i,1] >= 6:
            if set3[i,0] > set3[i,1]:
                sets_p1 = sets_p1 + 1
                sets_p2 = sets_p2 + 0
            elif set3[i,0] < set3[i,1]:
                sets_p1 = sets_p1 + 0
                sets_p2 = sets_p2 + 1

    #Asigmar puntos por partido ganado
    if sets_p1 > sets_p2:
        w_p1 = 50
        w_p2 = 0
    elif sets_p1 < sets_p2:
        w_p1 = 0
        w_p2 = 50
    else:
        w_p1 = 0
        w_p2 = 0
    
    #print(sets_p1,',',sets_p2)
    
    #Calcular puntaje base (partido, sets, games)
    pt_p1[i] = w_p1 + sets_p1*20 + (set1[i,0] + set2[i,0] + set3[i,0])*5
    pt_p2[i] = w_p2 + sets_p2*20 + (set1[i,1] + set2[i,1] + set3[i,1])*5
        

# Calcular ranking y aplicar handicap

In [ ]:
score = score_default.copy()
score = np.array(score)
ranking = np.zeros((pt_p1.shape[0],score.shape[0]))

for i in range(pt_p1.shape[0]):
    #Índices de jugadores
    idx_j1 = [j for j in range(len(players)) if players[j] == j1[i]]
    idx_j2 = [j for j in range(len(players)) if players[j] == j2[i]]
    idx_j3 = [j for j in range(len(players)) if players[j] == j3[i]]
    idx_j4 = [j for j in range(len(players)) if players[j] == j4[i]]
    
    #Handicap
    hcp_p1 = (score[idx_j3] + score[idx_j4]) / (score[idx_j1] + score[idx_j2])
    hcp_p2 = (score[idx_j1] + score[idx_j2]) / (score[idx_j3] + score[idx_j4])
    
    #Cálculo puntos por partido
    score[idx_j1] = score[idx_j1] + pt_p1[i]*hcp_p1 - pt_p2[i]*hcp_p2
    score[idx_j2] = score[idx_j2] + pt_p1[i]*hcp_p1 - pt_p2[i]*hcp_p2
    score[idx_j3] = score[idx_j3] - pt_p1[i]*hcp_p1 + pt_p2[i]*hcp_p2
    score[idx_j4] = score[idx_j4] - pt_p1[i]*hcp_p1 + pt_p2[i]*hcp_p2
    
    #print(int(pt_p1[i]*hcp_p1 - pt_p2[i]*hcp_p2),',', int(- pt_p1[i]*hcp_p1 + pt_p2[i]*hcp_p2))
    
    #Escribir ranking
    ranking[i,:] = np.rint(score)

In [ ]:
print(ranking)

In [ ]:
#Cuando hay empate en sets, los puntos que se generan dependen únicamente de la diferencia de games. Encontrar alternativa.
#Asegurar que nunca pase que el que gane pierda puntos
